In [1]:
import numpy as np

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_algorithms import NumPyMinimumEigensolver

from qiskit_algorithms.optimizers import SLSQP

from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator

In [3]:
from qiskit_nature.units import DistanceUnit
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian = problem.hamiltonian
second_q_op = hamiltonian.second_q_op()

In [4]:
mapper = JordanWignerMapper()
qubit_p_op = mapper.map(second_q_op)

In [6]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    num_spatial_orbitals = problem.num_spatial_orbitals, #2
    num_particles = problem.num_particles, # [1, 1]
    #excitations='sd',
    qubit_mapper=mapper,
    initial_state=HartreeFock(
        num_spatial_orbitals = problem.num_spatial_orbitals,
        num_particles = problem.num_particles, 
        qubit_mapper=mapper,
    ),
)

In [10]:
estimator = Estimator()
optimizer = SLSQP(maxiter=200)

vqe = VQE(Estimator(), ansatz, optimizer)
vqe.initial_point = [0] * ansatz.num_parameters
vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(qubit_p_op)
print(result)

{   'aux_operators_evaluated': None,
    'cost_function_evals': 25,
    'eigenvalue': -1.8572749339503598,
    'optimal_circuit': <qiskit_nature.second_q.circuit.library.ansatzes.uccsd.UCCSD object at 0x7fd1ec439f90>,
    'optimal_parameters': {   ParameterVectorElement(t[0]): -3.141949044549482,
                              ParameterVectorElement(t[1]): 8.250543147464255e-05,
                              ParameterVectorElement(t[2]): -3.253340585985715},
    'optimal_point': array([-3.14194904e+00,  8.25054315e-05, -3.25334059e+00]),
    'optimal_value': -1.8572749339503598,
    'optimizer_evals': None,
    'optimizer_result': <qiskit_algorithms.optimizers.optimizer.OptimizerResult object at 0x7fd1ed42dd50>,
    'optimizer_time': 0.23407387733459473}
